Temporal Innovator:
- Retrocausality project

In [ ]:
#%pip install mesa
#pip install ipykernel
#pip install ipywidgets --upgrade

In [1]:
from mesa import Agent, Model
from mesa.space import MultiGrid
import random

Initial setup:

In [9]:
class TimeAgent(Agent):
    def __init__(self, model):
        super().__init__(model) # Mesa 3: only model is passed
        self.model = model  # Explicitly store model reference if needed
        self.positions = [] # Will be populated with initial position in TimeModel

    def step(self):
        attempts = 0
        max_attempts = 10  # Prevent infinite loops
        while attempts < max_attempts:
            x, y = self.pos
            move = random.choice([(0, 1), (0, -1), (1, 0), (-1, 0)]) # Move randomly: up, down, left, right
            new_pos = (x + move[0], y + move[1])
            if (0 <= new_pos[0] < self.model.grid.width) and (0 <= new_pos[1] < self.model.grid.height):
                if new_pos not in self.model.occupied_positions or new_pos == self.pos:
                    self.model.grid.move_agent(self, new_pos)
                    self.model.occupied_positions.discard(self.pos) # Remove old position
                    self.model.occupied_positions.add(new_pos) # Add new position
                    self.positions.append(new_pos) # Record new position
                    break
            attempts += 1
        # If no valid move is found after max_attempts, agent stays put (position unchanged)

          
class TimeModel(Model):
    def __init__(self):
        super().__init__()
        self.grid = MultiGrid(10, 10, False)  # 10x10 grid, torus disabled
        self.schedule = []  # Manual agent list
        self.random = random.Random()
        self.step_count = 0  # Track step number
        self.occupied_positions = set()  # Track occupied positions
        
        # Create 5 agents with unique random starting positions
        available_positions = [(x, y) for x in range(10) for y in range(10)]  # All 10x10 positions
        self.random.shuffle(available_positions)  # Randomize order
        for i in range(5):
            agent = TimeAgent(self)
            start_pos = available_positions[i]  # Take a unique position
            self.grid.place_agent(agent, start_pos)
            agent.positions.append(start_pos)  # Record initial position
            self.occupied_positions.add(start_pos)
            self.schedule.append(agent)    

    def step(self):
        # Reset occupied positions for this step (will be rebuilt)
        self.occupied_positions.clear()
        for agent in self.schedule:
            self.occupied_positions.add(agent.pos)
        
        random.shuffle(self.schedule)  # Random activation
        for agent in self.schedule:
            agent.step()
        self.step_count += 1  # Increment before printing    
        # self.print_positions()
        

    # Print history positions of all agents
    def print_positions(self):
        sorted_agents = sorted(self.schedule, key=lambda a: a.unique_id)
        for agent in sorted_agents:
            print(f"Agent {agent.unique_id}: {agent.positions}")


model = TimeModel()

for _ in range(5):
    model.step()
model.print_positions()

Agent 1: [(8, 3), (7, 3), (7, 4), (6, 4), (7, 4), (6, 4)]
Agent 2: [(0, 1), (0, 2), (0, 3), (1, 3), (1, 4), (2, 4)]
Agent 3: [(5, 5), (5, 4), (5, 5), (6, 5), (6, 6), (6, 7)]
Agent 4: [(5, 3), (5, 2), (5, 1), (5, 2), (5, 3), (6, 3)]
Agent 5: [(7, 6), (7, 7), (8, 7), (7, 7), (8, 7), (9, 7)]
